In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
# from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# from sklearn import svm
import cv2

import av
import os
import shutil

%matplotlib inline

WORKING_FOLDER = "/home/eola/research"
DB_FOLDER = "{}/AFEW 5.0". format(WORKING_FOLDER)
FRAMES_FOLDER = "{}/Frames". format(WORKING_FOLDER)
DB_TRAIN = "{}/Train". format(DB_FOLDER)
DB_VAL = "{}/Val". format(DB_FOLDER)
FRAMES_VAL =  "{}/Val". format(FRAMES_FOLDER)

DICTIONARY_SIZE = 32 # cluster count

class_to_name = {0: 'Angry', 1: 'Sad', 2: 'Neutral', 3: 'Disgust', 4: 'Surprise', 5: 'Fear', 6: 'Happy'}
name_to_class = {'Angry': 0, 'Sad': 1, 'Neutral': 2, 'Disgust': 3, 'Surprise': 4, 'Fear': 5, 'Happy': 6}

In [2]:
def get_videos_dict():
    videos = {} #emotion_name: [] of videos names
    for root, dirs, files in os.walk(DB_TRAIN):
        # for all emotion folders
        for dirname in dirs:
            # put videos names in current emotion folder
            videos[dirname] = []
            for r, d, f in os.walk("{}/{}".format(DB_TRAIN, dirname)):
                videos[dirname].extend(f)
                break    
        # process root only
        break

#     for emotion, videos_list in videos.iteritems():
#         print "{}: {}".format(emotion, len(videos_list))
        
    return videos

In [3]:
def delete_frames():
    if os.path.exists(FRAMES_FOLDER):
        shutil.rmtree(FRAMES_FOLDER)

In [4]:
def extract_frames():
    print "extract_frames"
    
    if os.path.exists(FRAMES_FOLDER):
        print "Frames have already been extracted to {}".format(FRAMES_FOLDER)
        return
       
    videos = get_videos_dict()
    os.mkdir(FRAMES_FOLDER)
    for emotion_name, videos_list in videos.iteritems():
        print "{}: {}".format(emotion_name, len(videos_list))
        current_db_folder = "{}/{}".format(DB_TRAIN, emotion_name)
        current_frames_folder = "{}/{}".format(FRAMES_FOLDER, emotion_name)
        os.mkdir(current_frames_folder)
        video_count = 0
        for video in videos_list:
            video_frames_path = "{}/{}".format(current_frames_folder, video)
            os.mkdir(video_frames_path)
            container = av.open("{}/{}".format(current_db_folder, video))
            
            # extract frames amd save them as jpg files
            count = 0
            frames = []
            for packet in container.demux():
                for frame in packet.decode():
                    if(type(frame) == av.VideoFrame):
                        frame_path = "{}/{}.jpg".format(video_frames_path, count)
                        frame.to_image().save(frame_path)
#                         img = mpimg.imread(frame_path)
#                         plt.imshow(img)
#                         plt.show
                        count += 1
    
            video_count += 1
            if not video_count % 10:
                print video_count,
        print

In [5]:
# delete_frames()
extract_frames()

extract_frames
Frames have already been extracted to /home/eola/research/Frames


In [6]:
def get_frames_dict():
    frames = {} # {emotion_name: {video_name: [] of frames names}}
    for root, dirs, files in os.walk(FRAMES_FOLDER):
        # for all emotion folders
        for dirname in dirs:
            # put videos names in current emotion folder
            frames[dirname] = {}
            for vr, videodirs, vf in os.walk("{}/{}".format(FRAMES_FOLDER, dirname)):
                for videoname in videodirs:
                    frames[dirname][videoname] = []
                    for r, d, f in os.walk("{}/{}/{}".format(FRAMES_FOLDER, dirname, videoname)):
                        frames[dirname][videoname] .extend(f)
                        break                
                break    
        # process root only
        break

#     for emotion, videos_list in videos.iteritems():
#         print "{}: {}".format(emotion, len(videos_list))
        
    return frames

In [7]:
frames_dict = get_frames_dict()
# print frames

# print "number of emotions: {}".format(len(frames_dict.keys()))
# bow = cv2.BOWKMeansTrainer(len(frames_dict.keys()))
bow = cv2.BOWKMeansTrainer(DICTIONARY_SIZE)

sift = cv2.SIFT()
descriptors = []
for emotion_name, videos_dict in frames_dict.iteritems():
    print emotion_name
    for video_name, frames in videos_dict.iteritems():
#         print "{}: {} - {}".format(emotion_name, video_name, len(frames))
        for i in xrange(0, len(frames), 25):
#             print i
            frame = cv2.imread("{}/{}/{}/{}.jpg".format(FRAMES_FOLDER, emotion_name, video_name, i))
            gray = cv2.cvtColor(frame, cv2.CV_LOAD_IMAGE_GRAYSCALE)
            kp, dsc= sift.detectAndCompute(gray, None)
#             print "dsc size: {}".format(len(dsc))
            if len(kp):
                bow.add(dsc)
            else:
                print "no descriptors to add, kp len: {}".format(len(kp))
    print len(bow.getDescriptors())
            
dictionary = bow.cluster()
print len(dictionary)


#             print "number of descriptors: " + str(len(descriptors))
#             print descriptors
#         for i in range(,frames[emotion][])
# for path in training_paths:
#     image = cv2.imread(path)
#     print path
#     gray = cv2.cvtColor(image, cv2.CV_LOAD_IMAGE_GRAYSCALE)
#     kp, dsc= sift.detectAndCompute(gray, None)
#     descriptors.append(dsc)

# des = np.array(descriptors)

# k=5
# bow = cv2.BOWKMeansTrainer(k)
# bow.cluster(des)

Val
0
Angry


KeyboardInterrupt: 

In [8]:
import pickle

def save_dict(dictionary):
    output = open('dict.pkl', 'wb')
    pickle.dump(dictionary, output)
    output.close()
    
def load_dict():
    pkl_file = open('dict.pkl', 'rb')
    dictionary = pickle.load(pkl_file)
    pkl_file.close()
    return dictionary

In [9]:
bow_dict = load_dict()
print len(bow_dict)

frames_dict = get_frames_dict()

sift = cv2.SIFT()
bow_ext = cv2.BOWImgDescriptorExtractor(cv2.DescriptorExtractor_create("SIFT"), cv2.BFMatcher(cv2.NORM_L2))
bow_ext.setVocabulary(bow_dict)
histograms = {}
for emotion_name, videos_dict in frames_dict.iteritems():
    print emotion_name
    histograms[emotion_name] = []
    for video_name, frames in videos_dict.iteritems():
        for i in xrange(0, len(frames), 25):
            frame = cv2.imread("{}/{}/{}/{}.jpg".format(FRAMES_FOLDER, emotion_name, video_name, i))
            gray = cv2.cvtColor(frame, cv2.CV_LOAD_IMAGE_GRAYSCALE)
            kp, dsc= sift.detectAndCompute(gray, None)
            if len(kp):
                histogram = bow_ext.compute(gray, kp)[0]
                histograms[emotion_name].append(histogram)
            else:
                print "no descriptors, kp len: {}".format(len(kp))
            
    print len(histograms[emotion_name])
#             print len(histogram)
#             break

32
Val
0
Angry
346
Sad
343
Neutral
404
Disgust
238
Surprise
207
Fear
no descriptors, kp len: 0
207
Happy
407


In [10]:
print histograms.keys()

['Val', 'Angry', 'Sad', 'Neutral', 'Disgust', 'Surprise', 'Fear', 'Happy']


In [11]:
# i = 0
# for emotion_name in frames_dict.keys():
#     class_to_name[i] = emotion_name
#     name_to_class[emotion_name] = i
#     i += 1
print class_to_name
print name_to_class

# data = [[name_to_class[emotion_name]] * len(histogram[emotion_name]) for emotion_name in histograms.keys()]
# data = [[name_to_class[emotion_name]] * 5 for emotion_namse in histograms.keys()]
# join(data)
target = [name_to_class[emotion_name] for emotion_name in histograms.keys() for i in xrange(len(histograms[emotion_name]))]
print len(target)
data = [hist for hists in histograms.values() for hist in hists]
print len(data)

{0: 'Angry', 1: 'Sad', 2: 'Neutral', 3: 'Disgust', 4: 'Surprise', 5: 'Fear', 6: 'Happy'}
{'Angry': 0, 'Sad': 1, 'Neutral': 2, 'Disgust': 3, 'Surprise': 4, 'Fear': 5, 'Happy': 6}
2152
2152


In [81]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', C=50, gamma=10) # gamma=0.001, C=100
clf.fit(data, target)
clf.score(data, target)

0.65799256505576209

# Work with validation data

In [14]:
def get_videos_dict(path):
    videos = {} #emotion_name: [] of videos names
    for root, dirs, files in os.walk(path):
        # for all emotion folders
        for dirname in dirs:
            # put videos names in current emotion folder
            videos[dirname] = []
            for r, d, f in os.walk("{}/{}".format(path, dirname)):
                videos[dirname].extend(f)
                break    
        # process root only
        break

#     for emotion, videos_list in videos.iteritems():
#         print "{}: {}".format(emotion, len(videos_list))
        
    return videos

###############################################################################################

def extract_frames(db_path, frames_path):
    print "extract_frames"
    
    if os.path.exists(frames_path):
        print "Frames have already been extracted to {}".format(frames_path)
        return
       
    videos = get_videos_dict(db_path)
    os.mkdir(frames_path)
    for emotion_name, videos_list in videos.iteritems():
        print "{}: {}".format(emotion_name, len(videos_list))
        current_db_folder = "{}/{}".format(db_path, emotion_name)
        current_frames_folder = "{}/{}".format(frames_path, emotion_name)
        os.mkdir(current_frames_folder)
        video_count = 0
        for video in videos_list:
            video_frames_path = "{}/{}".format(current_frames_folder, video)
            os.mkdir(video_frames_path)
            container = av.open("{}/{}".format(current_db_folder, video))
            
            # extract frames amd save them as jpg files
            count = 0
            frames = []
            for packet in container.demux():
                for frame in packet.decode():
                    if(type(frame) == av.VideoFrame):
                        frame_path = "{}/{}.jpg".format(video_frames_path, count)
                        frame.to_image().save(frame_path)
#                         img = mpimg.imread(frame_path)
#                         plt.imshow(img)
#                         plt.show
                        count += 1
    
            video_count += 1
            if not video_count % 10:
                print video_count,
        print
        
###############################################################################################

def get_frames_dict(path):
    frames = {} # {emotion_name: {video_name: [] of frames names}}
    for root, dirs, files in os.walk(path):
        # for all emotion folders
        for dirname in dirs:
            # put videos names in current emotion folder
            frames[dirname] = {}
            for vr, videodirs, vf in os.walk("{}/{}".format(path, dirname)):
                for videoname in videodirs:
                    frames[dirname][videoname] = []
                    for r, d, f in os.walk("{}/{}/{}".format(path, dirname, videoname)):
                        frames[dirname][videoname] .extend(f)
                        break                
                break    
        # process root only
        break

#     for emotion, videos_list in videos.iteritems():
#         print "{}: {}".format(emotion, len(videos_list))
        
    return frames

###############################################################################################

def get_histograms(frames_path):
    bow_dict = load_dict()
    print len(bow_dict)

    frames_dict = get_frames_dict(frames_path)

    sift = cv2.SIFT()
    bow_ext = cv2.BOWImgDescriptorExtractor(cv2.DescriptorExtractor_create("SIFT"), cv2.BFMatcher(cv2.NORM_L2))
    bow_ext.setVocabulary(bow_dict)
    histograms = {}
    for emotion_name, videos_dict in frames_dict.iteritems():
        print emotion_name
        histograms[emotion_name] = []
        for video_name, frames in videos_dict.iteritems():
            for i in xrange(0, len(frames), 25):
                frame = cv2.imread("{}/{}/{}/{}.jpg".format(frames_path, emotion_name, video_name, i))
                gray = cv2.cvtColor(frame, cv2.CV_LOAD_IMAGE_GRAYSCALE)
                kp, dsc= sift.detectAndCompute(gray, None)
                if len(kp):
                    histogram = bow_ext.compute(gray, kp)[0]
                    histograms[emotion_name].append(histogram)
                else:
                    print "no descriptors, kp len: {}".format(len(kp))

        print len(histograms[emotion_name])
    return histograms

###############################################################################################

def split_hists(histograms):
    data = [hist for hists in histograms.values() for hist in hists]
    target = [name_to_class[emotion_name] for emotion_name in histograms.keys() for i in xrange(len(histograms[emotion_name]))]
    return data, target
    
###############################################################################################

# videos = get_videos_dict(DB_VAL)
extract_frames(DB_VAL, FRAMES_VAL)
# frames_hists = get_histograms(FRAMES_VAL)
print len(frames_hists['Angry'])

data_val, target_val = split_hists(frames_hists)
print len(target_val)
print len(data_val)
# clf.score(data_val, target_val)

extract_frames
Frames have already been extracted to /home/eola/research/Frames/Val
183
1083
1083


# Grid search for optimal SVM parameters

In [44]:
# params = [{'kernel': ['rbf'], 'gamma': [0.001, 0.01, 0.1, 1, 2, 3, 5, 10, 20, 50, 100], 'C': [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 100, 200, 300]},
#            {'kernel': ['poly'], 'gamma': [0.001, 0.01, 0.1, 1, 2, 3, 5, 10, 20, 50, 100], 'C': [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 100, 200, 300], 'degree': [3, 5, 7, 10, 14, 21]}]
params = [{'kernel': ['rbf'], 'gamma': [ 225, 250, 275, 300], 'C': [ 1, 2, 5, 7, 10, 13,]}]

print "Tuning SVM parameters"
clf = GridSearchCV(SVC(), params, cv=5,  scoring='accuracy')
clf.fit(data, target)

# print("Best parameters set found on development set:")
# print()
# print(clf.best_params_)
# print()
# print("Grid scores on development set:")
# print()
print "best params: {}".format(clf.best_params_)
print "all results:"
for params, mean_score, scores in clf.grid_scores_:
    print "{:5.3f} (+/- {:5.3f}) for {}".format(mean_score, scores.std() * 2, params)
#     print "%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params)
# print()

# print("Detailed classification report:")
# print()
# print("The model is trained on the full development set.")
# print("The scores are computed on the full evaluation set.")
# print()
# y_true, y_pred = y_test, clf.predict(X_test)
# print(classification_report(y_true, y_pred))
# print()

###############################################

# scores = ['precision', 'recall']

# for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     print()

#     clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
#                        scoring='%s_weighted' % score)
#     clf.fit(X_train, y_train)

#     print("Best parameters set found on development set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on development set:")
#     print()
#     for params, mean_score, scores in clf.grid_scores_:
#         print("%0.3f (+/-%0.03f) for %r"
#               % (mean_score, scores.std() * 2, params))
#     print()

#     print("Detailed classification report:")
#     print()
#     print("The model is trained on the full development set.")
#     print("The scores are computed on the full evaluation set.")
#     print()
#     y_true, y_pred = y_test, clf.predict(X_test)
#     print(classification_report(y_true, y_pred))
#     print()

Tuning SVM parameters
best params: {'kernel': 'rbf', 'C': 5, 'gamma': 250}
all results:
0.212 (+/- 0.033) for {'kernel': 'rbf', 'C': 1, 'gamma': 225}
0.211 (+/- 0.031) for {'kernel': 'rbf', 'C': 1, 'gamma': 250}
0.212 (+/- 0.032) for {'kernel': 'rbf', 'C': 1, 'gamma': 275}
0.216 (+/- 0.040) for {'kernel': 'rbf', 'C': 1, 'gamma': 300}
0.220 (+/- 0.034) for {'kernel': 'rbf', 'C': 2, 'gamma': 225}
0.218 (+/- 0.034) for {'kernel': 'rbf', 'C': 2, 'gamma': 250}
0.216 (+/- 0.037) for {'kernel': 'rbf', 'C': 2, 'gamma': 275}
0.218 (+/- 0.041) for {'kernel': 'rbf', 'C': 2, 'gamma': 300}
0.222 (+/- 0.061) for {'kernel': 'rbf', 'C': 5, 'gamma': 225}
0.224 (+/- 0.044) for {'kernel': 'rbf', 'C': 5, 'gamma': 250}
0.224 (+/- 0.044) for {'kernel': 'rbf', 'C': 5, 'gamma': 275}
0.224 (+/- 0.042) for {'kernel': 'rbf', 'C': 5, 'gamma': 300}
0.222 (+/- 0.060) for {'kernel': 'rbf', 'C': 7, 'gamma': 225}
0.223 (+/- 0.045) for {'kernel': 'rbf', 'C': 7, 'gamma': 250}
0.224 (+/- 0.043) for {'kernel': 'rbf', 'C':

# Score against validation data

In [42]:
clf = SVC(kernel='rbf', C=10, gamma=175)
clf.fit(data, target)
clf.score(data_val, target_val)

0.20036934441366575